In [ ]:
import os

os.chdir("../")

import matplotlib.pyplot as plt
import torch
from matplotlib import colormaps as cm

cmap = cm.get_cmap("viridis")

from pathlib import Path

import hydra

hydra.core.global_hydra.GlobalHydra.instance().clear()
hydra.initialize(version_base="1.3", config_path="../configs/")
cfg = hydra.compose(config_name="eval.yaml")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
root_dir = Path("logs/calibrate/runs")
ckpt_path = root_dir / "<experiment folder>/cmodel.ckpt"

model = hydra.utils.instantiate(cfg.model)
# load weights from checkpoint
checkpoint = torch.load(ckpt_path, map_location=device)
model.load_state_dict(checkpoint["state_dict"])
nc_curves = checkpoint["nc_curves"]
print(nc_curves.shape)
model = model.to(device)
print(model.device)

The cell below picks out certain pixel locations and plots the corresponding calibration curves.

In [ ]:
coords_x = torch.tensor([4, 20, 150, 160, 170, 300, 316])
coords_y = torch.tensor([4, 20, 120, 220, 236])
grid = torch.meshgrid(coords_x, coords_y, indexing="ij")

class_i = 1
img = torch.ones_like(nc_curves[0, 0])
img[grid[1], grid[0]] = 0

plt.imshow(img, cmap="gray", vmin=0, vmax=1)
plt.show()
plt.clf()

nc_curves_sub = nc_curves[:, class_i, grid[1], grid[0]]

sub_w = nc_curves_sub.shape[2]
sub_h = nc_curves_sub.shape[1]

fig, axes = plt.subplots(nrows=sub_w, ncols=sub_h, figsize=(10, 10))

# loop over the last two dimensions
for i in range(sub_w):
    for j in range(sub_h):
        ncss = nc_curves_sub[:, j, i]
        axes[i, j].plot(ncss)
        axes[i, j].set_ylim([0, 1])
        axes[i, j].set_xlim([0, ncss.shape[0]])
        axes[i, j].axis("off")

fig.show()